In [1]:
from helpers.helper_functions import read_and_set_df, set_datatypes, pd, np, msno, go, plt, sns

In [2]:
df = read_and_set_df('data/test.csv', train=True)

df.describe(include = 'all').transpose() #transpose = Tabelle transponieren für eine bessere Ansicht

df.head()

,"id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle__damage,annual__premium,policy_sales_channel,vintage"
0,381000$Male$68$1$28.0$1$1-2 Year$No$53066.0$12...
1,381001$Male$78$1$8.0$0$1-2 Year$Yes$28301.0$12...
2,381002$Male$22$1$28.0$1$< 1 Year$No$2630.0$153...
3,381003$Female$20$1$28.0$0$1-2 Year$Yes$38627.0...
4,381004$Female$44$1$28.0$1$1-2 Year$No$24984.0$...
